In [9]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu

In [24]:
df = pd.read_csv('/content/drive/MyDrive/ds/ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294480 entries, 0 to 294479
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294480 non-null  int64 
 1   timestamp     294480 non-null  object
 2   group         294480 non-null  object
 3   landing_page  294480 non-null  object
 4   converted     294480 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [79]:
df.shape

(286690, 5)

In [32]:
df.apply(lambda x: x.nunique())

,0
user_id,290585
timestamp,35993
group,2
landing_page,2
converted,2


In [34]:
df.isnull().sum()

,0
user_id,0
timestamp,0
group,0
landing_page,0
converted,0


In [36]:
df.drop_duplicates(subset = 'user_id', keep = False, inplace = True)
df.shape

(286690, 5)

In [42]:
amount = df.groupby(['group', 'landing_page']).agg({'landing_page':lambda x:x.value_counts()})
ratio = df.groupby(['group', 'landing_page']).agg({'converted':'mean'})


,,converted
group,landing_page,
control,old_page,0.120173
treatment,new_page,0.118726


In [70]:
control = df.loc[df['group'] == 'control', 'converted']
treatment = df.loc[df['group'] == 'treatment', 'converted']

# AB test


Сначала проверю на нормальность

H0: Выборки из нормального распределения

H1: Выборки не из нормального распределения

Буду основываться на значение p-value, если оно будет меньше 0.05, то H0 отклоняем.

In [72]:
p_value = shapiro(control).pvalue
p_value

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 143293.
  res = hypotest_fun_out(*samples, **kwds)


9.036752175762015e-178

In [73]:
p_value = shapiro(treatment).pvalue
p_value

/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 143397.
  res = hypotest_fun_out(*samples, **kwds)


6.263942656903057e-178

Тк p-value меньше 0.05 - H0 отклоняется


Проверю так же на гомогенность

H0: выборки гомогенны

H1: выборки не гомогенны

In [74]:
p_value = levene(control, treatment).pvalue
p_value

0.2322897281547632

p-value > 0.05 - H0 не отклоняем

Буду использовать непараметрический тест (Манна - Уитни), тк не соблюдается условие на нормальность

H0: между новой и старой версиями нет разницы

H1: между новой и старой версиями разница есть

In [77]:
p_value = mannwhitneyu(control, treatment).pvalue
p_value

0.23228910319572493

p-value > 0.05 - H0 не отвергаем, значит что разницы между разными версиями нет